In [1]:
import multiprocessing
import pandas as pd
import re

In [2]:
tables = pd.read_csv('output-data/statistics/expanded_tables_annotations_cpa.csv')

In [110]:
cpa_statistics = pd.read_csv('output-data/statistics/cpa_statistics.csv')
cpa_statistics = cpa_statistics.loc[(cpa_statistics['column_count'] >= 50) ]
cpa_statistics

,cpa_label,table_count,column_count,tables_and_densities
0,url,224945,224945,{'Product_corememoriesco.com_September2020.jso...
1,name,947672,947672,{'Product_corememoriesco.com_September2020.jso...
2,priceCurrency,771630,771630,{'Product_corememoriesco.com_September2020.jso...
3,price,725296,725296,{'Product_corememoriesco.com_September2020.jso...
4,description,761243,763396,{'Product_corememoriesco.com_September2020.jso...
...,...,...,...,...
210,publication,87,87,{'Book_montellolibrary.org_September2020.json....
213,seasonNumber,104,104,{'TVEpisode_animecons.tv_September2020.json.gz...
215,industry,54,54,{'JobPosting_bruinfinancial.com_September2020....
220,availabilityEnds,79,79,{'Product_civico.com_September2020.json.gz': {...


In [112]:
add_labels = cpa_statistics['cpa_label'].tolist()

In [5]:
selection = pd.read_csv('output-data/cpa-datasets/selected_1_2.csv')
selection['selected_cols_number'] = selection['selected_cols'].apply(lambda row: len(eval(row)))
selection_dict = selection.sort_values('selected_cols_number').to_dict('records')

selected_cols = {}
for row in selection_dict:
    selected_cols[row['file_name']] = eval(row['selected_cols'])

In [ ]:
#Select from already selected tables all non-annotated columns with a limit of 6500 columns
count_class = {} #Count columns per Schema.org type/class
rel_count = {} #Count columns per CPA label

for tab in selection_dict:
    sel = eval(tab['selected_cols'])
    rels = eval(tab['rel_labels'])
    
    if tab['class'] not in count_class:
        count_class[tab['class']] = 0
    
    #Select a maximum of 6500 columns per Schema.org type/class 
    if count_class[tab['class']] < 6500:
    
        for col in eval(tab['all_cols']):
            if col != 'name':
                #If column has not been yet selected:
                if col not in sel:
                    count_class[tab['class']] += 1
                    #Add to selected
                    selected_cols[tab['file_name']][col] = 'Random'

                    if rels[col] not in rel_count:
                        rel_count[rels[col]] = 0

                    rel_count[rels[col]] += 1


                else:
                    if rels[col] not in rel_count:
                        rel_count[rels[col]] = 0

                    rel_count[rels[col]] += 1

In [148]:
#Mark only labels that have not yet reached at least 100 examples
low_labels = [ x for x in rel_count if rel_count[x] < 100 ] + add_labels
low_labels = list(set(low_labels))

In [ ]:
#Add more examples to CTA labels that have not reached enough examples
count_class_2 = {} #Count columns per Schema.org type, do not pass 6500 columns

for cl in count_class:
    count_class_2[cl] = 0
    if count_class[cl] < 2000:
        d = tables.loc[(~tables['file_name'].isin(selected_cols)) & (tables['class'] == cl ) & (tables['overall_table_density'] >= 70 )]
        
        for index, row in d.iterrows():
            cpa = eval(row['rel_labels'])
            for col in eval(row['all_cols']):
                if col != 'name':
                    if rel_count_2[cpa[col]] < 2000:
                        if count_class_2[cl] < (6500 - count_class[cl]):
                            if row['file_name'] not in selected_cols:
                                selected_cols[row['file_name']] = {}
                            selected_cols[row['file_name']][col] = 'Random 2'
                            count_class_2[cl] += 1
        

In [151]:
#Select maximum 300 columns per CPA label
new_cols = {}
for l in low_labels:
    new_cols[l] = []
    
for index, row in tables.sort_values('column_count', ascending=False).iterrows():
    rels = eval(row['rel_labels'])
    
    for col in eval(row['all_cols']):
        if col != 'name' and rels[col] in low_labels and len(new_cols[rels[col]]) < 300:
                
            if row['file_name'] not in selected_cols or col not in selected_cols[row['file_name']]:
                
                new_cols[rels[col]].append(col+'_'+row['file_name'])
                
                #add to selected columns
                if row['file_name'] not in selected_cols:
                    selected_cols[row['file_name']] = {}
                selected_cols[row['file_name']][col] = 'Random 3'
          

## Assemble all selected columns

In [159]:
selection = tables.loc[ tables['file_name'].isin(selected_cols)]
sel_cols = []
for index, row in selection.iterrows():
    sel_cols.append(selected_cols[row['file_name']])
selection['selected_cols'] = sel_cols
selection

,class,file_name,column_count,number_of_rows,overall_table_density,all_cols,rel_labels,type_labels,sel_2
0,Product,Product_corememoriesco.com_September2020.json.gz,6,72,100,"{'url': 100, 'name': 100, 'offers:pricecurrenc...","{'url': 'url', 'name': 'name', 'offers:pricecu...","{'url': 'URL', 'name': 'Museum/name', 'offers:...","{'offers:price': 'Value Heterogeneity', 'url':..."
1,Event,Event_davidlohmueller.com_September2020.json.gz,7,20,88,"{'startdate': 100, 'name': 100, 'description':...","{'startdate': 'startDate', 'name': 'name', 'de...","{'startdate': 'DateTime', 'name': 'Museum/name...","{'startdate': 'Intra similarity', 'enddate': '..."
2,Product,Product_winehousegvl.com_September2020.json.gz,5,43,100,"{'name': 100, 'sku': 100, 'offers:pricecurrenc...","{'name': 'name', 'sku': 'sku', 'offers:pricecu...","{'name': 'Museum/name', 'sku': 'IdentifierAT',...",{'offers:price': 'Value Heterogeneity'}
4,Product,Product_juvenileplanet.com_September2020.json.gz,5,10,88,"{'name': 100, 'sku': 80, 'offers:price': 80, '...","{'name': 'name', 'sku': 'sku', 'offers:price':...","{'name': 'Museum/name', 'sku': 'IdentifierAT',...",{'offers:price': 'Value Heterogeneity'}
5,Product,Product_uppolomallets.com_September2020.json.gz,6,82,93,"{'name': 100, 'description': 90, 'offers:price...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',...",{'offers:price': 'Value Heterogeneity'}
...,...,...,...,...,...,...,...,...,...
947427,Place,Place_stevenspointjournal.com_September2020.js...,14,61,88,"{'name': 100, 'telephone': 100, 'branchcode': ...","{'name': 'name', 'telephone': 'telephone', 'br...","{'name': 'Museum/name', 'telephone': 'telephon...",{'branchcode': 'Random 3'}
947460,CreativeWork,CreativeWork_gsx-r1000.de_September2020.json.gz,8,12,95,"{'headline': 100, 'datemodified': 100, 'descri...","{'headline': 'headline', 'datemodified': 'date...","{'headline': 'Text', 'datemodified': 'DateTime...","{'datecreated': 'Value Heterogeneity', 'headli..."
947507,MusicAlbum,MusicAlbum_elviscarden.com_September2020.json.gz,5,21,99,"{'name': 100, 'numtracks': 100, 'track:inalbum...","{'name': 'name', 'numtracks': 'numTracks', 'tr...","{'name': 'Museum/name', 'numtracks': 'Integer'...","{'numtracks': 'Random 3', 'track:name': 'Rando..."
947553,CreativeWork,CreativeWork_ywbc.org.uk_September2020.json.gz,3,19,100,"{'datecreated': 100, 'name': 100, 'author:name...","{'datecreated': 'dateCreated', 'name': 'name',...","{'datecreated': 'DateTime', 'name': 'Museum/na...","{'datecreated': 'Inter Similarity', 'author:na..."


In [163]:
res = []
for index, row in selection.iterrows():
    rel_labels = eval(row['rel_labels'])
    type_labels = eval(row['type_labels'])
    densities = eval(row['all_cols'])
    
    for cols in selected_cols_4[row['file_name']]:
        if cols in rel_labels and cols in type_labels:
            res.append([row['class'], cols, row['file_name'], rel_labels[cols], type_labels[cols], densities[cols], selected_cols[row['file_name']][cols] ])
        

In [164]:
dataset = pd.DataFrame(res, columns=['class', 'column_name', 'file_name', 'relation_label', 'type_label', 'density', 'selection_type'])
dataset

,class,column_name,file_name,relation_label,type_label,density,selection_type
0,Product,offers:price,Product_corememoriesco.com_September2020.json.gz,price,price,100,Value Heterogeneity
1,Product,url,Product_corememoriesco.com_September2020.json.gz,url,URL,100,Random
2,Product,offers:pricecurrency,Product_corememoriesco.com_September2020.json.gz,priceCurrency,currency,100,Random
3,Product,offers:description,Product_corememoriesco.com_September2020.json.gz,description,Text,100,Random
4,Product,image,Product_corememoriesco.com_September2020.json.gz,image,URL,100,Random
...,...,...,...,...,...,...,...
163738,MusicAlbum,numtracks,MusicAlbum_elviscarden.com_September2020.json.gz,numTracks,Integer,100,Random 3
163739,MusicAlbum,track:name,MusicAlbum_elviscarden.com_September2020.json.gz,track,MusicRecording,100,Random 3
163740,CreativeWork,datecreated,CreativeWork_ywbc.org.uk_September2020.json.gz,dateCreated,DateTime,100,Inter Similarity
163741,CreativeWork,author:name,CreativeWork_ywbc.org.uk_September2020.json.gz,author,Person,100,Random


In [170]:
#Remove columns belonging to CTA labels that do not have a minimum of 50 examples
s = dataset.groupby(['relation_label'])['column_name'].count()
no_rels = list(s[s < 50].keys())
dataset = dataset.loc[~dataset['relation_label'].isin(no_rels)]
dataset

,class,column_name,file_name,relation_label,type_label,density,selection_type
0,Product,offers:price,Product_corememoriesco.com_September2020.json.gz,price,price,100,Value Heterogeneity
1,Product,url,Product_corememoriesco.com_September2020.json.gz,url,URL,100,Random
2,Product,offers:pricecurrency,Product_corememoriesco.com_September2020.json.gz,priceCurrency,currency,100,Random
3,Product,offers:description,Product_corememoriesco.com_September2020.json.gz,description,Text,100,Random
4,Product,image,Product_corememoriesco.com_September2020.json.gz,image,URL,100,Random
...,...,...,...,...,...,...,...
163738,MusicAlbum,numtracks,MusicAlbum_elviscarden.com_September2020.json.gz,numTracks,Integer,100,Random 3
163739,MusicAlbum,track:name,MusicAlbum_elviscarden.com_September2020.json.gz,track,MusicRecording,100,Random 3
163740,CreativeWork,datecreated,CreativeWork_ywbc.org.uk_September2020.json.gz,dateCreated,DateTime,100,Inter Similarity
163741,CreativeWork,author:name,CreativeWork_ywbc.org.uk_September2020.json.gz,author,Person,100,Random


In [178]:
dataset.to_csv('output-data/cpa-datasets/dataset.csv', index=False)